# Como adicionar docentes do seu programa na Wikidata

Olá!
Esse notebook documenta o processo utilizado no repositório Bioinfo Brasil para adicionar docentes de um programa de pós-graduação à Wikidata.
O passo a passo utiliza como exemplo o [programa de pós-graduação associado UTFPR/UFPR-CP](http://www.utfpr.edu.br/cursos/coordenacoes/stricto-sensu/ppgab/area-academica/docentes) mas, com pouco esforço, pode ser adaptado para outros programas.

O procedimento utiliza pacotes de [*web-scraping*](https://pt.wikipedia.org/wiki/Coleta_de_dados_web) e o [Google Planilhas](https://www.google.com/intl/pt-BR/sheets/about/), então recomenda-se ter alguma familiaridade com essas ferramentas.

Pacotes utilizados:
- [Requests](https://docs.python-requests.org/en/latest/)
- [Pandas](https://pandas.pydata.org/)
- [BeautifulSoup4](https://beautiful-soup-4.readthedocs.io/en/latest/)
- [lxml](https://pypi.org/project/lxml/)


In [ ]:
import re
import csv
import requests
import lxml.html

from bs4 import BeautifulSoup

Primeiro, vamos adquirir o conteudo da página

In [ ]:
url = "http://www.utfpr.edu.br/cursos/coordenacoes/stricto-sensu/ppgab/area-academica/docentes"

page = requests.get(url)

Em seguida, podemos parsear o conteúdo HTML da página. Essa parte requer um conhecimento precedente de *web scraping* para compreender de fato o que está acontecendo, mas em linhas gerais pode se dizer o seguinte:

- Utilizamos uma [Xpath](https://pt.wikipedia.org/wiki/XPath) para raspar todos os elementos na seção 'Docentes Permanentes' da página.
- Vendo que cada docente é representado por um elemento `<p>`, selecionamos todos os elementos p.

In [ ]:
tree = lxml.html.fromstring(page.content)

elements = "".join([
    lxml.html.tostring(el, method="html", encoding="unicode") for el 
    in tree.xpath('//div[@id="parent-fieldname-text"]/h2[last()]/preceding-sibling::*')
    ])

docentes = BeautifulSoup(elements, 'html.parser').find_all('p')

Em seguida, podemos escrever o conteúdo da raspagem em uma planilha CSV:
- O nome da professora, que adquirimos retirando o conteúdo adiante dos parênteses (que seguem o nome de cada uma, representando a instituição).
- O link para o currículo Lattes, que é representado pelos elementos `<a>`.

In [ ]:
with open("docentes.csv", "w") as csvfile:
    docente_writer = csv.writer(csvfile, dialect="unix")
    docente_writer.writerow(["docente", "lattes"])
    for row in docentes:
        if len(row.text.strip()) != 0:
            prof = re.sub(r"\(.*\)*", "", row.text).strip()
            lattes = row.find("a")["href"]
            docente_writer.writerow([prof, lattes])
